# Environment Settings

In [1]:
import torch
print(torch.version.cuda)


12.4


In [ ]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_id)
    gpu_capability = torch.cuda.get_device_capability(gpu_id)
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    
    print(f"Total number of GPU: {torch.cuda.device_count()}")  
    print(f"Total GPU memory: {total_memory / 1e9} GB")
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Compute Capability: {gpu_capability}")
else:
    print("No GPU is available.")

Total number of GPU: 4
Total GPU memory: 51.033931776 GB
GPU ID: 0
GPU Name: NVIDIA RTX A6000
GPU Compute Capability: (8, 6)


In [ ]:
import torch


torch.cuda.set_device(2)


print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Current Device: 2
Device Name: NVIDIA RTX A6000


device(type='cuda')

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('../dataset/national_illness.csv', index_col='date', parse_dates=True)

In [8]:
data.head()

,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 5-24,ILITOTAL,NUM. OF PROVIDERS,OT
date,,,,,,,
2002-01-01,1.22262,1.16668,582,805,2060,754,176569
2002-01-08,1.33344,1.21650,683,872,2267,785,186355
2002-01-15,1.31929,1.13057,642,878,2176,831,192469
2002-01-22,1.49484,1.25246,728,1045,2599,863,207512
2002-01-29,1.47195,1.30237,823,1189,2907,909,223208


In [9]:
print(data.dtypes)
print('shape: ', data.shape)

% WEIGHTED ILI       float64
%UNWEIGHTED ILI      float64
AGE 0-4                int64
AGE 5-24               int64
ILITOTAL               int64
NUM. OF PROVIDERS      int64
OT                     int64
dtype: object
shape:  (966, 7)


In [10]:
target = data['OT']
features = data.drop(['OT'],axis=1)

In [11]:
features['DATE'] = features.index.strftime('%Y%m%d%H').astype(int)

In [12]:
target.index

DatetimeIndex(['2002-01-01', '2002-01-08', '2002-01-15', '2002-01-22',
               '2002-01-29', '2002-02-05', '2002-02-12', '2002-02-19',
               '2002-02-26', '2002-03-05',
               ...
               '2020-04-28', '2020-05-05', '2020-05-12', '2020-05-19',
               '2020-05-26', '2020-06-02', '2020-06-09', '2020-06-16',
               '2020-06-23', '2020-06-30'],
              dtype='datetime64[ns]', name='date', length=966, freq=None)

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)                           
    np.random.seed(seed)                        
    torch.manual_seed(seed)                     
    torch.cuda.manual_seed(seed)                
    torch.cuda.manual_seed_all(seed)            
    torch.backends.cudnn.deterministic = True   
    torch.backends.cudnn.benchmark = False      

# Model Modification for DLinear (Decomposition-wise Look-ahead Augmentation)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class Model(nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.individual = configs.individual
        self.channels = configs.enc_in
        self.attach_to_trend = configs.attach_to_trend
        self.attach_to_seasonal = configs.attach_to_seasonal
        

        
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        

        
        self.seq_len_trend = self.seq_len + (self.pred_len // 3) if self.attach_to_trend else self.seq_len
        self.seq_len_seasonal = self.seq_len + (self.pred_len // 3) if self.attach_to_seasonal else self.seq_len

        
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

                
                
                
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
            
            
            

    def forward(self, x, ground_truth=None):
        
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if ground_truth is not None:
            kernel_size = 25
            self.decompsition = series_decomp(kernel_size)
            ground_truth_seasonal, ground_truth_trend = self.decompsition(ground_truth)

            
            if self.seq_len_seasonal > self.seq_len:
                ground_truth_seasonal=ground_truth_seasonal.permute(0, 2, 1)
                
                seasonal_init = torch.cat([seasonal_init, ground_truth_seasonal], dim=2)
            if self.seq_len_trend > self.seq_len:
                ground_truth_trend=ground_truth_trend.permute(0, 2, 1)
                
                trend_init = torch.cat([trend_init, ground_truth_trend], dim=2)
        
        
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) 

# Split the Data

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


target = data['OT'].values.reshape(-1, 1)  


seq_len  = 104
pred_len = 60


total_len  = len(target)
num_train  = int(total_len * 0.7)
num_test   = int(total_len * 0.2)
num_val    = total_len - num_train - num_test

border1s = [
    0,
    num_train - seq_len,
    total_len - num_test - seq_len
]
border2s = [
    num_train,
    num_train + num_val,
    total_len
]


train = target[border1s[0]:border2s[0]]
val   = target[border1s[1]:border2s[1]]
test  = target[border1s[2]:border2s[2]]


scaler = StandardScaler()
train = scaler.fit_transform(train)
val   = scaler.transform(val)
test  = scaler.transform(test)


def create_inout_sequences_univariate(data, seq_len, pred_len):
    seqs = []
    for i in range(len(data) - seq_len - pred_len + 1):
        seq_x = data[i : i + seq_len]
        seq_y = data[i + seq_len : i + seq_len + pred_len]
        seqs.append((seq_x, seq_y))
    return seqs

train_data = create_inout_sequences_univariate(train, seq_len, pred_len)
val_data   = create_inout_sequences_univariate(val,   seq_len, pred_len)
test_data  = create_inout_sequences_univariate(test,  seq_len, pred_len)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_sequences = torch.tensor([x[0] for x in train_data]).float().to(device)
train_labels    = torch.tensor([x[1] for x in train_data]).float().to(device)

val_sequences   = torch.tensor([x[0] for x in val_data]).float().to(device)
val_labels      = torch.tensor([x[1] for x in val_data]).float().to(device)

test_sequences  = torch.tensor([x[0] for x in test_data]).float().to(device)
test_labels     = torch.tensor([x[1] for x in test_data]).float().to(device)


print("Train Sequences:", train_sequences.shape)
print("Train Labels:   ", train_labels.shape)
print("Val Sequences:  ", val_sequences.shape)
print("Val Labels:     ", val_labels.shape)
print("Test Sequences: ", test_sequences.shape)
print("Test Labels:    ", test_labels.shape)


Train Sequences: torch.Size([513, 104, 1])
Train Labels:    torch.Size([513, 60, 1])
Val Sequences:   torch.Size([38, 104, 1])
Val Labels:      torch.Size([38, 60, 1])
Test Sequences:  torch.Size([134, 104, 1])
Test Labels:     torch.Size([134, 60, 1])


In [16]:
set_seed(42)

In [ ]:

batch_size = 32


train_dataset = TensorDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_dataset = TensorDataset(val_sequences, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


test_dataset = TensorDataset(test_sequences, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 1st Model Training (Early Stopping)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
import torch.optim as optim
import os


seq_len = 104
pred_len = 60  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.01, lradj='type1',  patience=3, save_path="./model_ILI", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), os.path.join(path, 'checkpoint_ES.pth'))
        self.val_loss_min = val_loss




def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')
        
        
        early_stopping(val_loss, model, args.save_path)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=10, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.02
Epoch 0 | Train Loss: 0.4560689737691599 | Val Loss: 0.1167852021753788
Validation loss decreased (inf --> 0.116785).  Saving model ...
Updating learning rate to 0.01
Epoch 1 | Train Loss: 0.14994261326158748 | Val Loss: 0.0750259980559349
Validation loss decreased (0.116785 --> 0.075026).  Saving model ...
Updating learning rate to 0.005
Epoch 2 | Train Loss: 0.10049126428716323 | Val Loss: 0.036533212289214134
Validation loss decreased (0.075026 --> 0.036533).  Saving model ...
Updating learning rate to 0.0025
Epoch 3 | Train Loss: 0.08887883604449384 | Val Loss: 0.04569799639284611
EarlyStopping counter: 1 out of 3
Updating learning rate to 0.00125
Epoch 4 | Train Loss: 0.0810554803294294 | Val Loss: 0.03402449283748865
Validation loss decreased (0.036533 --> 0.034024).  Saving model ...
Updating learning rate to 0.000625
Epoch 5 | Train Loss: 0.08088933194384855 | Val Loss: 0.03513266704976559
EarlyStopping counter: 1 out of 3
Updating learning rate t

# 1st Model Training (1 Epoch)

In [ ]:
import torch.optim as optim
import os



seq_len = 104
pred_len = 60  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.01, lradj='type1',  patience=3, save_path="./model_ILI", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    
    for epoch in range(epochs):

        if epoch == 1:
            torch.save(model.state_dict(), os.path.join(args.save_path, 'checkpoint_1.pth'))
            print("Model saved at epoch 1")

        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')

    torch.save(model.state_dict(), os.path.join(args.save_path, f'checkpoint_{epochs}.pth'))
    print(f"Model saved at epoch {epochs}")
        
        
        
        
        
        


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=1, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.02
Epoch 0 | Train Loss: 0.49339962794500236 | Val Loss: 0.2598186582326889
Model saved at epoch 1


# Perform Inference and Form Predicted Value (PV) Sequences

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


test_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in test_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    test_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    test_1st_results[ckpt_epoch] = {
        f'mse_test_E{ckpt_epoch}': mse,
        f'mae_test_E{ckpt_epoch}': mae,
        f'test_pv_E{ckpt_epoch}': test_pv
    }






[Predicting with model from epoch ES]
Epoch ES | MSE: 0.9693130254745483 | MAE: 0.866802453994751

[Predicting with model from epoch 1]
Epoch 1 | MSE: 1.5943959951400757 | MAE: 0.9945883750915527


In [22]:
test_1st_results.keys()

dict_keys(['ES', 1])

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


val_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in val_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    val_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    val_1st_results[ckpt_epoch] = {
        f'mse_val_E{ckpt_epoch}': mse,
        f'mae_val_E{ckpt_epoch}': mae,
        f'val_pv_E{ckpt_epoch}': val_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.03629432991147041 | MAE: 0.14875763654708862

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.24839092791080475 | MAE: 0.41603896021842957


In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


train_dataloader_infer = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


checkpoint_epochs = ['ES', 1]


train_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in train_dataloader_infer:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    train_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    train_1st_results[ckpt_epoch] = {
        f'mse_train_E{ckpt_epoch}': mse,
        f'mae_train_E{ckpt_epoch}': mae,
        f'train_pv_E{ckpt_epoch}': train_pv
    }




[Predicting with model from epoch ES]


Epoch ES | MSE: 0.08267417550086975 | MAE: 0.22074133157730103

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.24398237466812134 | MAE: 0.4001050889492035


In [25]:
print("train_sequences shape: ",train_sequences.shape)
print("val_sequences shape: ",val_sequences.shape)
print("test_sequences shape: ",test_sequences.shape)

print('---------------------------------------------')

print("train_pv shape: ",train_pv.shape)
print("val_pv shape: ",val_pv.shape)
print("test_pv shape: ",test_pv.shape)


print('---------------------------------------------')

print("train_labels shape: ",train_labels.shape)
print("val_labels shape: ",val_labels.shape)
print("test_labels shape: ",test_labels.shape)


train_sequences shape:  torch.Size([513, 104, 1])
val_sequences shape:  torch.Size([38, 104, 1])
test_sequences shape:  torch.Size([134, 104, 1])
---------------------------------------------
train_pv shape:  torch.Size([513, 60, 1])
val_pv shape:  torch.Size([38, 60, 1])
test_pv shape:  torch.Size([134, 60, 1])
---------------------------------------------
train_labels shape:  torch.Size([513, 60, 1])
val_labels shape:  torch.Size([38, 60, 1])
test_labels shape:  torch.Size([134, 60, 1])


# Segment Generation Function (sliding window-based)

In [ ]:
def create_sliding_segments(predictions, labels, segment_length, stride=1):
    """
    시퀀스에 슬라이딩 윈도우를 stride 간격으로 적용하여
    (B, num_segments, segment_length, C) 형태로 반환합니다.

    Args:
        predictions: Tensor of shape (B, T, C)
        labels: Tensor of shape (B, T, C)
        segment_length: int, 각 세그먼트의 길이 (예: pred_len // 3)
        stride: int, 슬라이딩 윈도우 stride 간격

    Returns:
        pred_segments: (B, num_segments, segment_length, C)
        label_segments: (B, num_segments, segment_length, C)
    """
    B, T, C = predictions.shape
    num_segments = (T - segment_length) // stride + 1

    pred_segments = []
    label_segments = []

    for i in range(0, T - segment_length + 1, stride):
        pred_seg = predictions[:, i:i+segment_length, :]  
        label_seg = labels[:, i:i+segment_length, :]      
        pred_segments.append(pred_seg.unsqueeze(1))       
        label_segments.append(label_seg.unsqueeze(1))

    pred_segments = torch.cat(pred_segments, dim=1)       
    label_segments = torch.cat(label_segments, dim=1)     

    return pred_segments, label_segments

# Segmentation for PV Sequence

In [ ]:
configs.pred_len 

60

In [ ]:
segment_len = configs.pred_len // 3
stride = 1  

train_segments_by_epoch = {}
val_segments_by_epoch = {}
test_segments_by_epoch = {}

for epoch in checkpoint_epochs:
    train_pv = train_1st_results[epoch][f'train_pv_E{epoch}']  
    val_pv = val_1st_results[epoch][f'val_pv_E{epoch}']        
    test_pv = test_1st_results[epoch][f'test_pv_E{epoch}']     

    
    train_pred_segments, train_label_segments = create_sliding_segments(train_pv, train_labels, segment_len,stride)
    val_pred_segments, val_label_segments = create_sliding_segments(val_pv, val_labels, segment_len, stride)
    test_pred_segments, test_label_segments = create_sliding_segments(test_pv, test_labels, segment_len, stride)

    train_segments_by_epoch[epoch] = {
        f'train_pred_segments_E{epoch}': train_pred_segments,  
        f'train_label_segments_E{epoch}': train_label_segments
    }

    val_segments_by_epoch[epoch] = {
        f'val_pred_segments_E{epoch}': val_pred_segments,
        f'val_label_segments_E{epoch}': val_label_segments
    }

    test_segments_by_epoch[epoch] = {
        f'test_pred_segments_E{epoch}': test_pred_segments,
        f'test_label_segments_E{epoch}': test_label_segments
    }

    
    print(f"[Epoch {epoch}] Train seg shape: {train_pred_segments.shape}, Val: {val_pred_segments.shape}, Test: {test_pred_segments.shape}")


[Epoch ES] Train seg shape: torch.Size([513, 41, 20, 1]), Val: torch.Size([38, 41, 20, 1]), Test: torch.Size([134, 41, 20, 1])
[Epoch 1] Train seg shape: torch.Size([513, 41, 20, 1]), Val: torch.Size([38, 41, 20, 1]), Test: torch.Size([134, 41, 20, 1])


## Save 1st Performance Metrics, PV, and Segment

In [ ]:
save_dir = './1st_results_ILI'  
os.makedirs(save_dir, exist_ok=True)


torch.save(train_1st_results, os.path.join(save_dir, 'train_1st_results.pt'))
torch.save(val_1st_results, os.path.join(save_dir, 'val_1st_results.pt'))
torch.save(test_1st_results, os.path.join(save_dir, 'test_1st_results.pt'))


torch.save(train_segments_by_epoch, os.path.join(save_dir, 'train_segments_by_epoch.pt'))
torch.save(val_segments_by_epoch, os.path.join(save_dir, 'val_segments_by_epoch.pt'))
torch.save(test_segments_by_epoch, os.path.join(save_dir, 'test_segments_by_epoch.pt'))

print("✅ All dictionaries saved successfully in ./saved_results")


✅ All dictionaries saved successfully in ./saved_results


# 2nd Model Training

In [30]:
train_segments_by_epoch = torch.load('./1st_results_ILI/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ILI/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ILI/test_segments_by_epoch.pt', weights_only=False)

In [31]:

print(train_segments_by_epoch.keys())
print(train_segments_by_epoch['ES'].keys())
print(train_segments_by_epoch[1].keys())

dict_keys(['ES', 1])
dict_keys(['train_pred_segments_EES', 'train_label_segments_EES'])
dict_keys(['train_pred_segments_E1', 'train_label_segments_E1'])


In [ ]:

train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']



## ES->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path
import time


epo = 'ES'

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 104 + pred_len//3,
    pred_len         = 60,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ILI_E{epo}_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()


for seg in range(len(train_pred_segments_ES[0])):
    print(f"\n=========== Segment {seg} ===========")
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_ES[:, seg, :, :]   
    va_pv = val_pred_segments_ES[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=32, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=32, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False  

        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True


            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()
        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()






=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 0] Training Time = 0.27 sec
[Segment 0] GPU Memory Start = 23.90 MB
[Segment 0] GPU Memory End   = 24.08 MB
[Segment 0] GPU Peak Memory  = 25.18 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 1] Training Time = 0.18 sec
[Segment 1] GPU Memory Start = 24.08 MB
[Segment 1] GPU Memory End   = 24.14 MB
[Segment 1] GPU Peak Memory  = 25.24 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 2] 

## 1E->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


epo = 1


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 104 + pred_len//3,
    pred_len         = 60,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ILI_E{epo}_1E/",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()



for seg in range(len(train_pred_segments_E1[0])):
    print(f"\n=========== Segment {seg} ===========")
    
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_E1[:, seg, :, :]   
    va_pv = val_pred_segments_E1[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=32, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=32, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False
        
        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True



            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()

        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()



=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 0] Training Time = 0.10 sec
[Segment 0] GPU Memory Start = 24.14 MB
[Segment 0] GPU Memory End   = 24.14 MB
[Segment 0] GPU Peak Memory  = 25.28 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 1] Training Time = 0.07 sec
[Segment 1] GPU Memory Start = 24.14 MB
[Segment 1] GPU Memory End   = 24.20 MB
[Segment 1] GPU Peak Memory  = 25.30 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0073 MB, total = 0.1245 MB
  pv per batch = 0.0024 MB, total = 0.0415 MB
[Segment 2] 

# 2nd Inference for Validation Set

In [35]:
train_segments_by_epoch = torch.load('./1st_results_ILI/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ILI/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ILI/test_segments_by_epoch.pt', weights_only=False)

In [ ]:
train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']


## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 60,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 41
batch_sz = 32

results2_ILI_ES_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()      
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_ES_val_1E[seg] = {
        "pred2_ES_val_1E"    : pred_all,   
        "true2_ES_val_1E"    : true_all,   
        "metrics2_ES_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_ES_val_1E[10]["metrics2_ES_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.13450181484222412  MAE:0.295777827501297
▶ seg0  mean  MSE:0.13450181484222412  MAE:0.295777827501297

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.13558247685432434  MAE:0.29071804881095886
▶ seg1  mean  MSE:0.13558247685432434  MAE:0.29071804881095886

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.12583646178245544  MAE:0.2841237783432007
▶ seg2  mean  MSE:0.12583646178245544  MAE:0.2841237783432007

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.144429013133049  MAE:0.3027746379375458
▶ seg3  mean  MSE:0.144429013133049  MAE:0.3027746379375458

―――― Inference  |  Segment 4 ――――


  seg4-ch0  MSE:0.13516013324260712  MAE:0.2916949987411499
▶ seg4  mean  MSE:0.13516013324260712  MAE:0.2916949987411499

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.154254749417305  MAE:0.31717121601104736
▶ seg5  mean  MSE:0.154254749417305  MAE:0.31717121601104736

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.16183383762836456  MAE:0.32474851608276367
▶ seg6  mean  MSE:0.16183383762836456  MAE:0.32474851608276367

―――― Inference  |  Segment 7 ――――
  seg7-ch0  MSE:0.17384812235832214  MAE:0.34599095582962036
▶ seg7  mean  MSE:0.17384812235832214  MAE:0.34599095582962036

―――― Inference  |  Segment 8 ――――
  seg8-ch0  MSE:0.18394657969474792  MAE:0.35794854164123535
▶ seg8  mean  MSE:0.18394657969474792  MAE:0.35794854164123535

―――― Inference  |  Segment 9 ――――
  seg9-ch0  MSE:0.1404646933078766  MAE:0.3026638925075531
▶ seg9  mean  MSE:0.1404646933078766  MAE:0.3026638925075531

―――― Inference  |  Segment 10 ――――
  seg10-ch0  MSE:0.14492139220237732  MAE:0.30710324645

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 60,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 41
batch_sz = 32

results2_ILI_E1_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_E1_val_1E[seg] = {
        "pred2_E1_val_1E"    : pred_all,   
        "true2_E1_val_1E"    : true_all,   
        "metrics2_E1_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_E1_val_1E[10]["metrics2_E1_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.22225278615951538  MAE:0.38295239210128784
▶ seg0  mean  MSE:0.22225278615951538  MAE:0.38295239210128784

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.16887857019901276  MAE:0.31902292370796204
▶ seg1  mean  MSE:0.16887857019901276  MAE:0.31902292370796204

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.1531093865633011  MAE:0.30952000617980957
▶ seg2  mean  MSE:0.1531093865633011  MAE:0.30952000617980957

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.1828872412443161  MAE:0.350271612405777
▶ seg3  mean  MSE:0.1828872412443161  MAE:0.350271612405777

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.1665547788143158  MAE:0.314331978559494
▶ seg4  mean  MSE:0.1665547788143158  MAE:0.314331978559494

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.1403600573539734  MAE:0.3018302917480469
▶ seg5  mean  MSE:0.1403600573539734  MAE:0.3018302917480469

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.17847977578

# 2nd Inference for Test Set

## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 60
seq_len = 104 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 60,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 41
batch_sz = 32

results2_ILI_ES_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_ES_test_1E[seg] = {
        "pred2_ES_test_1E"    : pred_all,   
        "true2_ES_test_1E"    : true_all,   
        "metrics2_ES_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_ES_test_1E[10]["metrics2_ES_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:1.0728267431259155  MAE:0.8444520235061646
▶ seg0  mean  MSE:1.0728267431259155  MAE:0.8444520235061646

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:1.0734931230545044  MAE:0.8472680449485779
▶ seg1  mean  MSE:1.0734931230545044  MAE:0.8472680449485779

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:1.0476515293121338  MAE:0.8317326307296753
▶ seg2  mean  MSE:1.0476515293121338  MAE:0.8317326307296753

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.723545491695404  MAE:0.5954878926277161
▶ seg3  mean  MSE:0.723545491695404  MAE:0.5954878926277161

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.9890325665473938  MAE:0.7780251502990723
▶ seg4  mean  MSE:0.9890325665473938  MAE:0.7780251502990723

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.9814160466194153  MAE:0.7606054544448853
▶ seg5  mean  MSE:0.9814160466194153  MAE:0.7606054544448853

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.9945473670959473  M

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 60
seq_len = 104 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 60,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 41
batch_sz = 32

results2_ILI_E1_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_E1_test_1E[seg] = {
        "pred2_E1_test_1E"    : pred_all,   
        "true2_E1_test_1E"    : true_all,   
        "metrics2_E1_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_E1_test_1E[10]["metrics2_E1_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:1.1483793258666992  MAE:0.8569033741950989
▶ seg0  mean  MSE:1.1483793258666992  MAE:0.8569033741950989

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:1.0996233224868774  MAE:0.8329564929008484
▶ seg1  mean  MSE:1.0996233224868774  MAE:0.8329564929008484

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:1.0024055242538452  MAE:0.7986404299736023
▶ seg2  mean  MSE:1.0024055242538452  MAE:0.7986404299736023

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:1.339106559753418  MAE:0.9412488341331482
▶ seg3  mean  MSE:1.339106559753418  MAE:0.9412488341331482

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.8926180005073547  MAE:0.7334471940994263
▶ seg4  mean  MSE:0.8926180005073547  MAE:0.7334471940994263

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:1.0460789203643799  MAE:0.8114387392997742
▶ seg5  mean  MSE:1.0460789203643799  MAE:0.8114387392997742

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.9660854339599609  M

# Rank Models (by 2nd validation performance)

In [45]:
results1_ILI_train = torch.load("./1st_results_ILI/train_1st_results.pt")
results1_ILI_val = torch.load("./1st_results_ILI/val_1st_results.pt")
results1_ILI_test = torch.load("./1st_results_ILI/test_1st_results.pt")

### ES->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 41
SEG_CNT     = 41
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ILI_ES_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ILI_ES_val_1E[seg]["metrics2_ES_val_1E"]:
            mse_table[seg, ch] = results2_ILI_ES_val_1E[seg]["metrics2_ES_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_ES_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ILI_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_ES_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_ES_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.248391

  Top  1 | seg 21 | MSE 0.121254 (+0.127137)
  Top  2 | seg 02 | MSE 0.125836 (+0.122554)
  Top  3 | seg 14 | MSE 0.126257 (+0.122134)
  Top  4 | seg 39 | MSE 0.128763 (+0.119628)
  Top  5 | seg 11 | MSE 0.132124 (+0.116267)
  Top  6 | seg 23 | MSE 0.133246 (+0.115145)
  Top  7 | seg 16 | MSE 0.134367 (+0.114024)
  Top  8 | seg 00 | MSE 0.134502 (+0.113889)
  Top  9 | seg 04 | MSE 0.135160 (+0.113231)
  Top 10 | seg 01 | MSE 0.135582 (+0.112808)
  Top 11 | seg 20 | MSE 0.135616 (+0.112775)
  Top 12 | seg 09 | MSE 0.140465 (+0.107926)
  Top 13 | seg 40 | MSE 0.141970 (+0.106421)
  Top 14 | seg 25 | MSE 0.142187 (+0.106204)
  Top 15 | seg 32 | MSE 0.143693 (+0.104698)
  Top 16 | seg 03 | MSE 0.144429 (+0.103962)
  Top 17 | seg 10 | MSE 0.144921 (+0.103470)
  Top 18 | seg 24 | MSE 0.148479 (+0.099912)
  Top 19 | seg 17 | MSE 0.154012 (+0.094379)
  Top 20 | seg 05 | MSE 0.154255 (+0.094136)
  Top 21 | seg 19 |

### 1E->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 41
SEG_CNT     = 41
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ILI_E1_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ILI_E1_val_1E[seg]["metrics2_E1_val_1E"]:
            mse_table[seg, ch] = results2_ILI_E1_val_1E[seg]["metrics2_E1_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_E1_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ILI_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_E1_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_E1_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.248391

  Top  1 | seg 17 | MSE 0.129054 (+0.119337)
  Top  2 | seg 20 | MSE 0.137411 (+0.110980)
  Top  3 | seg 05 | MSE 0.140360 (+0.108031)
  Top  4 | seg 16 | MSE 0.144653 (+0.103738)
  Top  5 | seg 08 | MSE 0.144861 (+0.103530)
  Top  6 | seg 29 | MSE 0.147936 (+0.100455)
  Top  7 | seg 24 | MSE 0.148829 (+0.099562)
  Top  8 | seg 33 | MSE 0.150440 (+0.097951)
  Top  9 | seg 37 | MSE 0.151116 (+0.097275)
  Top 10 | seg 02 | MSE 0.153109 (+0.095282)
  Top 11 | seg 25 | MSE 0.154386 (+0.094005)
  Top 12 | seg 18 | MSE 0.157865 (+0.090526)
  Top 13 | seg 09 | MSE 0.160702 (+0.087689)
  Top 14 | seg 26 | MSE 0.161162 (+0.087229)
  Top 15 | seg 04 | MSE 0.166555 (+0.081836)
  Top 16 | seg 27 | MSE 0.166834 (+0.081557)
  Top 17 | seg 40 | MSE 0.167090 (+0.081301)
  Top 18 | seg 32 | MSE 0.167121 (+0.081270)
  Top 19 | seg 01 | MSE 0.168879 (+0.079512)
  Top 20 | seg 13 | MSE 0.170091 (+0.078300)
  Top 21 | seg 39 |

# Ensemble

## ES->1E


In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 41
CH_CNT   = 1
batch_sz = 32

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)   


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []




for ch in range(CH_CNT):
    print(f"\n=== Channel {ch} ===")

    
    
    
    v_mse = results1_ILI_test['ES']['mse_test_EES']
    v_mae = results1_ILI_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    for seg in top_seg_idx_ES_val_1E[ch][:TOP_K]:
        ckpt = f"./model2_ILI_EES_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] ckpt missing : seg{seg}")
            continue

        pv_seg = test_pred_segments_ES[:, seg, :, :]
        dl     = DataLoader(TensorDataset(test_sequences, test_labels, pv_seg),
                            batch_size=batch_sz, shuffle=False)

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        preds_test_arr.append(infer_one(mdl, dl, ch))   
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list, rho_list = [], []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min)/(V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min)/(R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1
    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]

    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)

    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    print(f"[Var-Corr]  K={best_K}")
    print(f"  MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"  MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")




if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE     : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%) : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE     : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%) : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.969313 | MAE : 0.866802


  — Var / |Corr| / Score —
   K= 5 | Var=2.737e-01 | |ρ|=0.7893 | S=1.0000 
   K=10 | Var=2.812e-01 | |ρ|=0.7815 | S=0.8845 <- pick
   K=15 | Var=2.905e-01 | |ρ|=0.7784 | S=0.9258 
   K=20 | Var=2.958e-01 | |ρ|=0.7845 | S=1.1728 
   K=25 | Var=3.085e-01 | |ρ|=0.7735 | S=1.0376 
   K=30 | Var=3.159e-01 | |ρ|=0.7703 | S=1.0525 
   K=35 | Var=3.347e-01 | |ρ|=0.7582 | S=0.9722 
   K=40 | Var=3.456e-01 | |ρ|=0.7538 | S=1.0000 
[Var-Corr]  K=10
  MSE = 0.795559 | Δ = +0.173754 | Δ% = +17.93%
  MAE = 0.740499 | Δ = +0.126303 | Δ% = +14.57%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE : 0.969313
Ensemble mean MSE : 0.795559
Average Δ MSE     : +0.173754
Average Δ MSE (%) : +17.93%

Vanilla  mean MAE : 0.866802
Ensemble mean MAE : 0.740499
Average Δ MAE     : +0.126303
Average Δ MAE (%) : +14.57%


## 1E->1E


In [49]:
results1_ILI_test['ES']['mae_test_EES']

0.866802453994751

In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 41
CH_CNT   = 1
batch_sz = 32

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)    



van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []





for ch in range(CH_CNT):

    print(f"\n=== Channel {ch} ===")

    
    v_mse = results1_ILI_test['ES']['mse_test_EES']
    v_mae = results1_ILI_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    
    
    for seg in top_seg_idx_E1_val_1E[ch][:TOP_K]:

        ckpt = f"./model2_ILI_E1_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] missing checkpoint : seg{seg}")
            continue

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        pv_seg = test_pred_segments_E1[:, seg, :, :]
        dl = DataLoader(
            TensorDataset(test_sequences, test_labels, pv_seg),
            batch_size=batch_sz, shuffle=False
        )

        preds_test_arr.append(infer_one(mdl, dl, ch))
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list = []
    rho_list = []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)   

        
        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        
        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R    = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    
    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min) / (V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min) / (R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1.1   

    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]
    best_K = Ks[int(np.argmin(scores))]

    
    
    
    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)  
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)
    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    sign = "+" if delta_mse > 0 else "-"
    print(f"[Var-Corr]  K={best_K}")
    print(f"   MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"   MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")





if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE  : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE  : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE      : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%)  : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE  : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE  : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE      : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%)  : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.969313 | MAE : 0.866802
  — Var / |Corr| / Score —
   K= 5 | Var=3.552e-01 | |ρ|=0.7556 | S=0.9732 
   K=10 | Var=3.505e-01 | |ρ|=0.7540 | S=0.8148 <- pick
   K=15 | Var=3.496e-01 | |ρ|=0.7580 | S=0.9580 
   K=20 | Var=3.546e-01 | |ρ|=0.7577 | S=1.0468 
   K=25 | Var=3.524e-01 | |ρ|=0.7615 | S=1.1553 
   K=30 | Var=3.660e-01 | |ρ|=0.7548 | S=1.1524 
   K=35 | Var=3.840e-01 | |ρ|=0.7416 | S=0.9790 
   K=40 | Var=3.999e-01 | |ρ|=0.7343 | S=1.0000 
[Var-Corr]  K=10
   MSE = 0.882557 | Δ = +0.086756 | Δ% = +8.95%
   MAE = 0.804579 | Δ = +0.062224 | Δ% = +7.18%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE  : 0.969313
Ensemble mean MSE  : 0.882557
Average Δ MSE      : +0.086756
Average Δ MSE (%)  : +8.95%

Vanilla  mean MAE  : 0.866802
Ensemble mean MAE  : 0.804579
Average Δ MAE      : +0.062224
Average Δ MAE (%)  : +7.18%
